# HW6 

# Hannah Wang

## Problem 1

### 1(a) 

$$\begin{aligned}
Cov(X_T) &= \Sigma Cov(W_T) \Sigma^T\\
    &= \begin{bmatrix} 0.3 & 0 \\ 0 & 0.2 \end{bmatrix} \begin{bmatrix} 1T & 0.8T \\ 0.8T & 1T \end{bmatrix} \begin{bmatrix} 0.3 & 0 \\ 0 & 0.2 \end{bmatrix} \\
    &= \begin{bmatrix} 0.09T & 0.048T \\ 0.048T & 0.04T \end{bmatrix}
\end{aligned}$$

### 1(b)

In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
class MultiGBM:
    
    def __init__(self,S0,r,correlations,sigma):
        self.S0 = S0
        self.r = r
        self.correlations = correlations
        self.sigma = sigma

In [3]:
hw6p1dynamics = MultiGBM(S0=np.array([100,110]),r=0.05,
                         correlations = np.array([[1, 0.8], [0.8, 1]]),  #You fill this in with a 2x2 correlation matrix
                         sigma = np.diag([0.3, 0.2]))

In [4]:
class CallOnBasket:
    
    def __init__(self,K,T,weights):
        self.K = K
        self.T = T
        self.weights = weights

In [5]:
hw6p1contract=CallOnBasket(K=110,T=1.0,weights = np.array([1/2, 1/2]))

In [6]:
class MC:
    
    def __init__(self, M, antithetic, control, seed):
        self.M = M                                  # How many simulations 
        self.antithetic = antithetic
        self.control = control
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible
        
    def price_callonbasket_multiGBM(MC,contract,dynamics):
        
        # You complete the coding of this function.
        # self.rng.multivariate_normal may be useful.
        # See documentation for numpy.random.Generator.multivariate_normal
        # as self.rng is an instance of numpy.random.Generator
        
        # You are not required to support the case where MC.control = MC.antithetic = True
        # (simultaneous use of control variate and antithetic)
        # But you are required to support the other 3 possible settings of MC.antithetic/MC.control 
        # namely False/False, True/False, False/True.
        # (ordinary MC, antithetic without control, control without antithetic)
        
        K, T, weights = contract.K, contract.T, contract.weights
        S0, r, corr, sigma = dynamics.S0, dynamics.r, dynamics.correlations, dynamics.sigma
        M, antithetic, control = MC.M, MC.antithetic, MC.control
    
        X_0 = (np.ones((M, len(S0)))* np.log(S0)).T
        sigma_mat = (np.ones((M, len(S0))) * np.diag(sigma)).T
    
        dW_T = MC.rng.multivariate_normal([0,0],corr,M) * np.sqrt(T)
        X_T = X_0 + (r-sigma_mat**2/2)*T + sigma.dot(dW_T.T)

        S_T = np.exp(X_T)
        H_T = weights.dot(S_T)
        C_T = np.exp(-r*T) * np.maximum(H_T-K, 0)
    
        call_price = np.mean(C_T)
        standard_error = np.std(C_T, ddof=1) / np.sqrt(M)
        
        def BScallPrice(sigma,S,rGrow,r,K,T):
            F=S*np.exp(rGrow*T)
            sd = sigma*np.sqrt(T)
            d1 = np.log(F/K)/sd+sd/2
            d2 = d1-sd
            return np.exp(-r*T)*(F*norm.cdf(d1)-K*norm.cdf(d2))
    
        if control:
            sig = np.sqrt(sigma[0,0]**2 + sigma[1,1]**2 + 2*sigma[0,0]*sigma[1,1]*corr[0,1])/2
            S = (S0[0]*S0[1])**0.5
            rGrow = r + (2*sigma[0,0]*sigma[1,1]*corr[0,1] - sigma[0,0]**2 - sigma[1,1]**2)/8
        
            C_BS = BScallPrice(sig,S,rGrow,r,K,T)
        
            G_Tcon = np.exp(np.mean(X_T, axis=0))
            C_Tcon = np.exp(-r*T) * np.maximum(G_Tcon-K, 0)
        
            cov = np.cov(C_T, C_Tcon)
            beta = cov[0,1] / cov[1,1]
            Y_con = C_T + beta*(C_BS - C_Tcon)
            call_price = np.mean(Y_con)
            standard_error = np.std(Y_con, ddof=1) / np.sqrt(M)
        
        return(call_price, standard_error)    

In [7]:
hw6p1bMC=MC(M=10000,antithetic=False,control=False,seed=0)
(call_price_ordinary, std_err_ordinary) = hw6p1bMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_ordinary, std_err_ordinary)

9.8581037987066 0.16800488136614875


### 1(c) 

In [8]:
hw6p1cMC=MC(M=10000,antithetic=True,control=False,seed=0)
(call_price_AV, std_err_AV) = hw6p1cMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_AV, std_err_AV)

9.930408770857396 0.09477429058267407


### 1(d) 

We know that, 

$$\begin{aligned}
    d \log{S_t} &= \frac{1}{S_t} (rS_tdt + \sigma S_t dW_t) - \frac{1}{2S_t^2} (\sigma^2 S_t^2 dt) \\
    &= (r-\frac{\sigma^2}{2})dt + \sigma dW_t \\
\end{aligned}$$

Then,

$$\begin{aligned}
    \log G_t &= \frac{1}{2} \bigg( \log S_t^{[1]} + \log S_t^{[2]}   \bigg)\\
    &= \frac{1}{2} \bigg(\log S_0^{[1]} + (r-\frac{\sigma_{[1]}^2}{2})t + \sigma_{[1]} W_t^{[1]} + \log S_0^{[2]} + (r-\frac{\sigma_{[2]}^2}{2})t + \sigma_{[2]} W_t^{[2]} \bigg) \\
    &= \frac{1}{2} \bigg(\log \big(S_0^{[1]}  S_0^{[2]} \big) + \big(2r-\frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{2} \big)t + \sigma_{[1]} W_t^{[1]} + \sigma_{[2]} W_t^{[2]} \bigg) \\
\end{aligned}$$

Thus, we can get,

$$\begin{aligned}
\mathbb{E} \big[ \log G_T \big] &= \frac{1}{2} \bigg(\log \big(S_0^{[1]}  S_0^{[2]} \big) + \big(2r-\frac{\sigma_{[1]}^2 + \sigma_{[2]}^2}{2} \big)T \bigg) \\
    Var \big[ \log G_T \big] &= Var \bigg[ \frac{1}{2} \big(\sigma_{[1]} W_t^{[1]} + \sigma_{[2]} W_t^{[2]} \big) \bigg] \\
    &= \frac{1}{4} Var \bigg[ \big(\sigma_{[1]} W_t^{[1]} + \sigma_{[2]} W_t^{[2]} \big) \bigg] \\
    &= \frac{1}{4} \bigg( \sigma_{[1]}^2 + \sigma_{[2]}^2 + 2 \sigma_{[1]} \sigma_{[2]} \rho_{[1],[2]} \bigg) T \\
\end{aligned}$$

### 1(e) 

$$\begin{aligned}
    C^G =C^{BS}((S_0^{[1]}S_0^{[2]})^{\frac{1}{2}},0,K,T,r + \frac{2\sigma_{[1]}\sigma_{[2]}\rho_{[1],[2]} - \sigma_{[1]}^2 - \sigma_{[2]}^2}{8},r,\frac{\sqrt{\sigma_{[1]}^2 + \sigma_{[2]}^2 + 2 \sigma_{[1]} \sigma_{[2]} \rho_{[1],[2]}}}{2} )
\end{aligned}$$

### 1(f) 

In [9]:
hw6p1fMC=MC(M=10000,antithetic=False,control=True,seed=0)
(call_price_CV, std_err_CV) = hw6p1fMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_CV, std_err_CV)

9.993510290823446 0.00447372947133335


## Problem 2

In [10]:
class GBM:
    
    def __init__(self,sigma,r,S0):
        self.sigma = sigma
        self.r = r
        self.S0 = S0

In [11]:
hw6p2dynamics=GBM(sigma=0.2,r=0.02,S0=100)

In [12]:
class CallOption:
    
    def __init__(self,K,T):
        self.K=K
        self.T=T

In [13]:
hw6p2contract=CallOption(K=150,T=1)

In [14]:
class MCimportance:
    
    def __init__(self, M, lamb, seed):
        self.M = M                                  # How many simulations 
        self.lamb = lamb                            # drift adjustment 
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible

    def price_call_GBM(self, contract,dynamics):
        
        # You complete the coding of this function.
        # self.rng.normal may be useful.
        # See documentation for numpy.random.Generator.normal
        # as self.rng is an instance of numpy.random.Generator
        
        K, T = contract.K, contract.T
        S0, r, sigma = dynamics.S0, dynamics.r, dynamics.sigma
        M, lamb = self.M, self.lamb
        
        W_1 = self.rng.normal(0,T,M)        
        X_T = np.log(S0) + (r-sigma**2/2)*T + sigma*(W_1 + lamb*T)
        S_T = np.exp(X_T)
        Y_T = np.exp(-r*T - lamb*W_1 - 0.5*T*lamb**2) * np.maximum(S_T-K, 0)

        call_price = np.mean(Y_T)
        standard_error = np.std(Y_T, ddof=1) / np.sqrt(M)

        return(call_price, standard_error)


### 2(a) 

In [15]:
hw6p2aMC=MCimportance(M=100000,lamb=0,seed=0) #zero drift adjustment gives ordinary MC

(call_price_ordinary, std_err_ordinary) =  hw6p2aMC.price_call_GBM(hw6p2contract,hw6p2dynamics)
print(call_price_ordinary, std_err_ordinary)

0.25270332833609405 0.007609293292996182


### 2(b) 

$$\begin{aligned}
    d{W}_t &= dW_t^* + \lambda dt \\
    dS_t &= r S_t dt + \sigma S_t (dW_t^* + \lambda dt) \\
    &= (r+\sigma \lambda)S_t dt + \sigma S_t dW_t^* \\
    {E}^* [S_t] &= S_0 e^{(r+\sigma \lambda)t} \\
    \lambda &= \frac{\log (\frac{\mathbb{E}^* [S_t]}{S_0}) / t - r}{\sigma}
\end{aligned}$$

In [21]:
(np.log(165/100)/1 - 0.02) / 0.2

2.4038764395624455

### 2(c) 

In [22]:
hw6p2cMC=MCimportance(M=100000,lamb=2.4038764395624455,seed=0) # Fill in the lamb parameter with the lambda that you compute in (b) 
(call_price_importsamp, std_err_importsamp) =  hw6p2cMC.price_call_GBM(hw6p2contract,hw6p2dynamics)
print(call_price_importsamp, std_err_importsamp)

0.24843662621391502 0.0007734271968138013
